In [241]:
import csv
import requests 

In [242]:
# the matrix

with open('/home/faculty/millbr02/pub/un-debates-augmented-1.csv', 'r', encoding='latin-1') as f:
    csv_iter = csv.reader(f)
    db = []
    for row in csv_iter:
        db.append(row)

    # complied list of country names
    ctryNames = []
    for entry in db[1:]:
        if entry[4] not in ctryNames:
            ctryNames.append(entry[4])

    # creating the matrix
    matrix = {x: {} for x in ctryNames}
    for country in matrix:
        matrix[country] = {x:0 for x in ctryNames}
    
    # populating the matrix
    for entry in db[1:]:
        for country in ctryNames:
            num = entry[3].count(country)
            othersMentioned = matrix.get(entry[4])
            othersMentioned[country] += num

In [243]:
# how many times each country mentioned itself and by others

mentionedDict = {}
for country in ctryNames:
    mentionedDict[country] = 0
    for speakingCtry in matrix:
        if country in matrix[speakingCtry]:
            mentionedDict[country] += matrix[speakingCtry][country]

In [244]:
# creating a dictionary of country codes and their corresponding coordinates 

res = requests.get("https://opendata.socrata.com/api/views/mnkm-8ram/rows.json")
j = res.json() 

latLongDict = {}
latLongCtryCodes = []
for line in j['data']:
    latLongDict[line[10]] = {'lat':line[12],'long':line[13]}
    latLongCtryCodes.append(line[10])

In [245]:
# appending latitude and longitude to each line in db

for line in db[1:]:
    if line[2] in latLongCtryCodes:
        line.append(latLongDict[line[2]]['lat'])
        line.append(latLongDict[line[2]]['long'])

In [246]:
# writing out a new file, format: [code,name,mentions,latitude,longitude]

with open('new_db.csv', 'w', encoding = 'latin-1') as outFile:
    outFile.write('code,name,mentions,latitude,longitude\n')
    ctryAdds = []
    for line in db[1:]:
        if line[2] not in ctryAdds and line[2] in latLongDict:
            ctryAdds.append(line[2])
            outFile.write("{0},{1},{2},{3},{4}\n".format(line[2],line[4],mentionedDict[line[4]],line[7],line[8]))
        elif line[2] not in ctryAdds and line[2] not in latLongDict:
            ctryAdds.append(line[2])
            outFile.write("{0},{1},{2}\n".format(line[2],line[4],mentionedDict[line[4]])) 

AttributeError: '_io.TextIOWrapper' object has no attribute 'wrie'